# Section 5.5.2
# Animate head in strip of land due to sudden change of boundary

Use the solution for the decline of head in a strip of land within parallel ditches to animat it.

Animation is done with `matplotlib.anamation.FuncAnimation`. This requires two functions. One that initializes the lines to be drawn and the other to update its data in an iteration loop. The latter function takes a single integer input that is the frame number within the iteration.

It makes sense to define a class, in this case "Strip" which has a method to compute itself. Doing this, the class is instantiated with all its data before running the animation, so that the aninamtion only requires the counter.

@TO 2020-12-09

### Strip formula


$$s(x, t) = h_0 \, \mbox{erfc}\left(\sqrt{\frac{x^2 S}{4 kD t}}\right)$$

$$s(x, t) = h_0 \, \mbox{erfc}\left(u\right),\,\,\,\,\,u=\sqrt{\frac{x^2 S}{4 kD t}}$$

$$\mbox{erfc}\left(z\right) = \frac 2 {\sqrt{\pi}} \intop_z^\infty e^{-y^2}
dy$$

$$\frac{d \mbox{erfc}(z)}{dz} = -\frac 2 {\sqrt{\pi}} e^{-z^2}
$$

$$Q_{x,t} = - kD \frac{\partial s}{\partial x} = h_0\,\sqrt{\frac {kDS}{\pi t}} \exp\left(-\frac{x^2 S}{4 kD t}\right)
$$

$$Q_{0,t} = h_0 \, \sqrt{\frac{kD S}{\pi t}}$$

$$s(x, t) = h_0 \sum_{i=0}^\infty 
\left\{
(-1)^i
\mbox{erfc}\left(\left[\left(i + \frac 1 2 \right) L + x\right]\sqrt{\frac S {4 kD t}}\right)
+
\mbox{erfc}\left(
              \left[\left(i + \frac 1 2 \right) L - x\right]\sqrt{\frac S {4 kD t}}\right)
\right\}
$$



$$s(x, t) = h_0 \sum_{i=0}^\infty 
\left\{
(-1)^i
\mbox{erfc}\left(\left[\left(i + \frac 1 2 \right) L + x\right]\sqrt{\frac S {4 kD t}}\right)
+
\mbox{erfc}\left(
              \left[\left(i + \frac 1 2 \right) L - x\right]\sqrt{\frac S {4 kD t}}\right)
\right\}
$$

In [14]:
attribs = lambda obj: [o for o in dir(obj) if not o.startswith('_')]

import numpy as np
from scipy.special import erfc
from matplotlib import pyplot as plt
from matplotlib import animation, rc
from matplotlib.animation import FuncAnimation
from matplotlib.patches import PathPatch, Path
from IPython.display import HTML
import pdb


def newfig(title='title', xlabel='xlabel', ylabel='ylabel',
                   xlim=None, ylim=None, size_inches=(12, 8)):
    fig, ax = plt.subplots()
    fig.set_size_inches(size_inches)
    ax.set_title(title)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    if xlim:
        ax.set_xlim(xlim)
    if ylim:
        ax.set_ylim(ylim)
    ax.grid()
    return fig, ax

In [34]:
# First set up the figure, the axis, and the plot element we want to animate
class Strip:
    
    def __init__(self, b=None, h0=None, kD=None, S=None, L=None, n=20):
        """Return a strip object.
        
        Parameters
        ----------
        b: float
            half-width of the strip
        h0: float
            initial head above drainage base
        kD: float
            transmissivity
        S: float
            Storage coefficient
        n: number of points in the half-strip
        """
        self.h0 = h0
        self.kD = kD
        self.S = S
        self.n = n
        self.x = np.linspace(0, 2 * L, n + 1)
        
    def compute(self, t, n=20):
        u = self.x * np.sqrt(self.S / (4 * self.kD * t))
        return self.h0 * erfc(u)
    
    def plot(self, times=None, n=20, ax=None):
        """Plot the head for all times."""
        for t in times:
            s = self.compute(t)
            ax.plot(self.x, s, label='t= {:.3f} d'.format(t))
            
    def tchar(self, x):
        """Return characterictic time for distance x."""
        return x ** 2 * self.S / (4 * self.kD)

In [40]:
data = {'kD': 600, 'S':0.2, "h0":2, "L": 150, "n":20}

strip = Strip(**data)

title = "Animation of head in infinite half strip with initial head increase at x=0.\n" +\
        'kD={:.0f} m2/d, S={:.2f}, h0={:.2f} m'\
        .format(*[data[k] for k in ['kD', 'S', 'h0']])

fig, ax = newfig(title, "x [m]", "head / water table [m]",
                  xlim=(0, 2 * data['L']), ylim=(0, data['h0']),
                  size_inches=(12, 8))

line, = ax.plot([], [], lw=2)
text  = ax.text(0.05, 0.90, '', fontsize=14, transform=ax.transAxes)
xchar = 50
ax.text(0.05, 0.8, 'Head change, tchar(x={:.0f} m) = {:.3f} d'
                    .format(xchar, strip.tchar(x=xchar)),
                    fontsize=14, transform=ax.transAxes)

n = 20  # total time 6 times the halftime
m = 10 # subdivision of each halftime
tchar_times = strip.tchar(xchar) * np.arange(n)
times    = strip.tchar(xchar) * np.arange(0, n, 1/m)[1:]

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return line, # receiver expects a tuple of the artists involved

# animation function.  This is called sequentially
def animate(t, strip, t50times, xchar, ax):
    # global declaration not strictly needed, but nice to make clear
    global line, text
    tchar = strip.tchar(xchar)
    y = strip.compute(t)
    line.set_data(strip.x, y)
    text.set_text('time = {:8.3f} d, t_rel = {:.1f}'.format(t, t/tchar))
    
    # Leave line behind at every tchar multiple.
    if t in tchar_times:
        ax.plot(strip.x, strip.compute(t), label='t / tchar = {:.1f}'.format(t / tchar))
    if t == tchar_times[-1]:
        ax.legend()
    return line, # receiver expects a tuple of the artitsts involved

# call the animator.  blit=True means only re-draw the parts that have changed.
print("Patience, computing and generating video takes about 30 sec. on mac...")
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=times, fargs=(strip, tchar_times, xchar, ax),
                               interval=20, blit=True, repeat=False)

plt.close(anim._fig)

Patience, computing and generating video takes about 30 sec. on mac...


In [41]:
# save the animation as an mp4.  This requires ffmpeg or mencoder to be
# installed.  The extra_args ensure that the x264 codec is used, so that
# the video can be embedded in html5.  You may need to adjust this for
# your system: for more information, see
# http://matplotlib.sourceforge.net/api/animation_api.html
anim.save('Land_intruded_from_left.mp4', fps=20, extra_args=['-vcodec', 'libx264'])
print(anim.save_count, " frames saved.")

199  frames saved.


In [42]:
out = HTML(anim.to_html5_video())
out

# Simulating the water intrusion in its aquifer setting (full picture)

The graph is put in the picture with the aquifer and the aquiclude and the lake water, exactly as it was drawn by hand. This makes a live picture of what is happening.
Instead of suddenly letting the initial water level rise, we will let the opaquecy increase from zero to 1. So that the new water level simply appears.

Of course, you could do it perfectly, using the other solutions as can be demonstrated for the Lake Nasser example further down.

In [100]:
pD = [[400, 25]]
pA = ax0.transData.transform(pD)
pF = ax0.figure.transFigure.inverted().transform(pA)
print(pD)
print(pA)
print(pF)
bbox2figure(0, hlake1, L1, hlake2, ax=ax0)

[[400, 25]]
[[267948.   8208.]]
[[310.125  19.   ]]


In [233]:
def bbox2figure(x0=None, y0=None, h=None, w=None, ax=None):
    """Convert axes data position to axes fraction position."""
    xy = [[x0, y0], [x0 + h, y0 + w]]
    uvDisp = ax0.transData.transform(xy) # Data --> Display
    uvFig  = ax0.figure.transFigure.inverted().transform(uvDisp) # Display to figure
    return uvFig[0][0], uvFig[0][1], uvFig[1][0] - uvFig[0][0], uvFig[1][1] - uvFig[0][1]


Aquif = dict(A=20., kD=600, S=0.2)

u = lambda x, t : x * np.sqrt(Aquif['S'] / (4 * Aquif['kD'] * t))

def rect(x0=None, y0=None, w=None, h=None, **kw):
    return PathPatch(Path([[x0, y0],
                         [x0 + w, y0],
                         [x0 + w, y0 + h],
                         [x0, y0 + h],
                         [x0, y0]], codes=[1, 2, 2, 2, 79]), **kw)
                
hbase, haquif, haclude, hlake1, hlake2 = 5, 25, 30, 30, 20  
L0, L1 = 150, 2000

params = {'base':  dict(x0=-L0, y0=0, w=L0 + L1, h=-hbase, fc='darkgreen'),
          'aquif': dict(x0=  0, y0=0, w=L1, h=haquif, fc='gold'),
          'aclud': dict(x0=  0, y0=haquif, w=L1, h=haclude, fc='darkgreen'),
          'lake1': dict(x0=-L0, y0=0, w=L0, h=hlake1, fc='blue'),
          'lake2': dict(x0=-L0, y0=hlake1, w=L0, h=hlake2, fc='blue')
         }

base    = rect(**params['base'])
aquif   = rect(**params['aquif'])
aclud   = rect(**params['aclud'])
lake1   = rect(**params['lake1'])
lake2   = rect(**params['lake2'])

things = [base, aquif, aclud, lake1, lake2]

fig, ax0 = plt.subplots(); fig.set_size_inches(12, 6)

for thing in things:
    ax0.add_patch(thing)
    
ax0.set_xlim((-L0, L1))
ax0.set_ylim((-hbase, haquif + haclude + 10))
ax0.set_fc("none")
bbox = dict(boxstyle="round", fc="0.8")
ax0.annotate('x', (0, 0.1 * haquif),
            ha='right', xytext=(L0, 0.1 * haquif), textcoords='data', va='center',
            fontsize=15, arrowprops=dict(arrowstyle='<-'))
ax0.annotate('', (-L0/2, hlake1),
            xytext=(-L0/2, hlake1 + hlake2), textcoords='data', va='center',
            fontsize=15, arrowprops=dict(arrowstyle='<->'))
ax0.text(-L0/2, hlake1 + 0.5 * hlake2, 'A', ha='center', fontsize=15, bbox=bbox)

ax0.text(3 * L0, 0.6 * haquif, 'kD={:.0f} m2/d, S={:.2f} [-]'.format(Aquif['kD'], Aquif['S']), fontsize=15)

x = np.linspace(0, L1, 400)

frac = np.linspace(-0.15,  1 ,115)[1:]
tmax = 100. # d
times  =  frac * tmax
alphas  =  np.fmax(0, 1 - np.exp(-frac * 10))
hlake = np.ones_like(times) * hlake2 


ax1 = fig.add_subplot(position=bbox2figure(0, hlake1, L1, hlake2, ax=ax0))
ax1.set_xlim((0, L1))
ax1.set_ylim((0, hlake2))
ax1.set_fc("none")
ax1.xaxis.set_visible(False)
ax1.yaxis.set_visible(False)

line, = ax1.plot(x, Aquif['A'] * erfc(u(x, 0.01)), color='c', zorder=100)
txt = ax0.text(0, haquif + haclude + 5, "t = {:6.1f} d".format(0), fontsize=15, bbox=bbox)

def init():
    lake2.set_visible(False)
    line.set_data([], [])
    txt.set_text("")
    return lake2, line, txt
def animate(talpha):
    t, alpha = talpha
    if t>0:
        lake2.set_visible(True)
    lake2.set_alpha(0.1)
    line.set_data(x, Aquif['A'] * erfc(u(x, np.fmax(0, t))))
    txt.set_text("t = {:6.1f} d".format(t))
    return lake2, line, txt

if True:
    anim = FuncAnimation(fig, animate, frames=zip(times, alphas), fargs=None, init_func=init,
                        interval=100, blit=True, repeat=False)

    plt.close(anim._fig)
    #out = HTML(anim.to_html5_video())
#out

In [234]:
fname = "StripWithIntrudingWater"

FFwriter = animation.FFMpegWriter(fps=30, extra_args=['-vcodec', 'libx264'], bitrate=1000)

anim.save(fname + '.mp4', fps=10, extra_args=['-vcodec', 'libx264'], bitrate=1000)
print(anim.save_count, " frames saved.") # Shows the number of frames saved.

/Users/Theo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in double_scalars
/Users/Theo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in multiply


100  frames saved.


In [235]:
!ffmpeg -i StripWithIntrudingWater.mp4 StripWithIntrudingWater.gif

ffmpeg version 3.2.1 Copyright (c) 2000-2016 the FFmpeg developers
  built with llvm-gcc 4.2.1 (LLVM build 2336.11.00)
  configuration: --prefix=/Volumes/Ramdisk/sw --enable-gpl --enable-pthreads --enable-version3 --enable-libspeex --enable-libvpx --disable-decoder=libvpx --enable-libmp3lame --enable-libtheora --enable-libvorbis --enable-libx264 --enable-avfilter --enable-libopencore_amrwb --enable-libopencore_amrnb --enable-filters --enable-libgsm --enable-libvidstab --enable-libx265 --disable-doc --arch=x86_64 --enable-runtime-cpudetect
  libavutil      55. 34.100 / 55. 34.100
  libavcodec     57. 64.101 / 57. 64.101
  libavformat    57. 56.100 / 57. 56.100
  libavdevice    57.  1.100 / 57.  1.100
  libavfilter     6. 65.100 /  6. 65.100
  libswscale      4.  2.100 /  4.  2.100
  libswresample   2.  3.100 /  2.  3.100
  libpostproc    54.  1.100 / 54.  1.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'StripWithIntrudingWater.mp4':
  Metadata:
    major_brand     : isom
    minor_version

# Sudden drawdown of lake level

In [253]:
Aquif = dict(A=15., kD=600, S=0.2)

u = lambda x, t : x * np.sqrt(Aquif['S'] / (4 * Aquif['kD'] * t))

def rect(x0=None, y0=None, w=None, h=None, **kw):
    return PathPatch(Path([[x0, y0],
                         [x0 + w, y0],
                         [x0 + w, y0 + h],
                         [x0, y0 + h],
                         [x0, y0]], codes=[1, 2, 2, 2, 79]), **kw)
                
hbase, haquif, hlake1, hlake2 = 5, 50, 30, Aquif['A']  
L0, L1 = 150, 2000

params = {'base':  dict(x0=-L0, y0=0, w=L0 + L1, h=-hbase, fc='darkgreen'),
          'aquif': dict(x0=  0, y0=0, w=L1, h=haquif, fc='gold'),
          'lake1': dict(x0=-L0, y0=0, w=L0, h=hlake1, fc='blue'),
          'lake2': dict(x0=-L0, y0=hlake1, w=L0, h=hlake2, fc='blue')
         }

base    = rect(**params['base'])
aquif   = rect(**params['aquif'])
lake1   = rect(**params['lake1'])
lake2   = rect(**params['lake2'])

things = [base, aquif, lake1, lake2]

fig, ax0 = plt.subplots(); fig.set_size_inches(12, 6)

for thing in things:
    ax0.add_patch(thing)
    
ax0.set_xlim((-L0, L1))
ax0.set_ylim((-hbase, haquif + 10))
ax0.set_fc("none")
bbox = dict(boxstyle="round", fc="0.8")
ax0.annotate('x', (0, 0.1 * haquif),
            ha='right', xytext=(L0, 0.1 * haquif), textcoords='data', va='center',
            fontsize=15, arrowprops=dict(arrowstyle='<-'))
ax0.annotate('', (-L0/2, hlake1),
            xytext=(-L0/2, hlake1 + hlake2), textcoords='data', va='center',
            fontsize=15, arrowprops=dict(arrowstyle='<->'))
ax0.text(-L0/2, hlake1 + 0.5 * hlake2, 'A', ha='center', fontsize=15, bbox=bbox)

ax0.text(3 * L0, 0.6 * haquif, 'kD={:.0f} m2/d, S={:.2f} [-]'.format(Aquif['kD'], Aquif['S']), fontsize=15)

x = np.linspace(0, L1, 400)

frac = np.linspace(-0.15,  1 ,115)[1:]
tmax = 100. # d
times  =  frac * tmax
alphas  =  np.fmax(0, 1 - np.exp(-frac * 10))
hlake = np.ones_like(times) * hlake2 


ax1 = fig.add_subplot(position=bbox2figure(0, hlake1, L1, hlake2, ax=ax0))
ax1.set_xlim((0, L1))
ax1.set_ylim((hlake2, 0))
ax1.set_fc("none")
ax1.set_frame_on(False)
ax1.xaxis.set_visible(False)
ax1.yaxis.set_visible(False)

line, = ax1.plot(x, Aquif['A'] * erfc(u(x, 0.01)), color='c', zorder=100)
txt = ax0.text(0, haquif + 5, "t = {:6.1f} d".format(0), fontsize=15, bbox=bbox)

aquif.set_sketch_params(scale=2, length=128, randomness=64)

def init():
    lake2.set_visible(True)
    line.set_data([], [])
    txt.set_text("")
    return lake2, line, txt
def animate(talpha):
    t, alpha = talpha
    if t>0:
        lake2.set_visible(False)
    lake2.set_alpha(0.1)
    line.set_data(x, Aquif['A'] * erfc(u(x, np.fmax(0, t))))
    txt.set_text("t = {:6.1f} d".format(t))
    return lake2, line, txt

if True:
    anim = FuncAnimation(fig, animate, frames=zip(times, alphas), fargs=None, init_func=init,
                        interval=100, blit=True, repeat=False)

    plt.close(anim._fig)
    #out = HTML(anim.to_html5_video())
#out

In [254]:
fname = "StripWithSuddenDDown"

FFwriter = animation.FFMpegWriter(fps=30, extra_args=['-vcodec', 'libx264'], bitrate=1000)

anim.save(fname + '.mp4', fps=10, extra_args=['-vcodec', 'libx264'], bitrate=1000)
print(anim.save_count, " frames saved.") # Shows the number of frames saved.

/Users/Theo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  This is separate from the ipykernel package so we can avoid doing imports until
/Users/Theo/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in multiply
  This is separate from the ipykernel package so we can avoid doing imports until


100  frames saved.


In [255]:
!ffmpeg -i StripWithSuddenDDown.mp4 StripWithSuddenDDown.gif

ffmpeg version 3.2.1 Copyright (c) 2000-2016 the FFmpeg developers
  built with llvm-gcc 4.2.1 (LLVM build 2336.11.00)
  configuration: --prefix=/Volumes/Ramdisk/sw --enable-gpl --enable-pthreads --enable-version3 --enable-libspeex --enable-libvpx --disable-decoder=libvpx --enable-libmp3lame --enable-libtheora --enable-libvorbis --enable-libx264 --enable-avfilter --enable-libopencore_amrwb --enable-libopencore_amrnb --enable-filters --enable-libgsm --enable-libvidstab --enable-libx265 --disable-doc --arch=x86_64 --enable-runtime-cpudetect
  libavutil      55. 34.100 / 55. 34.100
  libavcodec     57. 64.101 / 57. 64.101
  libavformat    57. 56.100 / 57. 56.100
  libavdevice    57.  1.100 / 57.  1.100
  libavfilter     6. 65.100 /  6. 65.100
  libswscale      4.  2.100 /  4.  2.100
  libswresample   2.  3.100 /  2.  3.100
  libpostproc    54.  1.100 / 54.  1.100
Input #0, mov,mp4,m4a,3gp,3g2,mj2, from 'StripWithSuddenDDown.mp4':
  Metadata:
    major_brand     : isom
    minor_version   